# Flask を使った簡易Webアプリ

まずは必要なライブラリをインストール

In [1]:
!pip install -r requirements.txt

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached flask_cors-5.0.1-py3-none-any.whl.metadata (961 bytes)
  Using cached SpeechRecognition-3.14.1-py3-none-any.whl.metadata (31 kB)
  Using cached openai-1.65.2-py3-none-any.whl.metadata (27 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached Flask_SocketIO-5.5.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


## その１　とりあえず音声ファイルをアップロードできるようにする．

In [42]:
%%writefile static/index.html

<!DOCTYPE html>
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Voice Chat App</title>
  </head>
  <body>
    <h1>Voice Chat App</h1>
    <button id="start">開始</button>
    <button id="stop" disabled>停止</button>
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
      document.addEventListener("DOMContentLoaded", () => {
        const startButton = document.getElementById("start");
        const stopButton = document.getElementById("stop");
        const transcriptionElement = document.getElementById("transcription");
        const aiResponseElement = document.getElementById("aiResponse");
        let mediaRecorder;
        let audioChunks = [];

        startButton.addEventListener("click", async () => {
          const stream = await navigator.mediaDevices.getUserMedia({
            audio: true,
          });
          mediaRecorder = new MediaRecorder(stream);

          mediaRecorder.ondataavailable = (event) => {
            audioChunks.push(event.data);
          };

          mediaRecorder.onstop = async () => {
            const audioBlob = new Blob(audioChunks, { type: "audio/webm" });
            audioChunks = [];

            const formData = new FormData();
            formData.append("audio", audioBlob, "recording.webm");

            fetch("/upload", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                transcriptionElement.textContent =
                  data.text || "認識できませんでした。";
                aiResponseElement.textContent =
                  data.ai_response || "AIの応答なし。";
              })
              .catch((error) => {
                console.error("Upload failed:", error);
                transcriptionElement.textContent = "エラーが発生しました。";
                aiResponseElement.textContent = "";
              });
          };

          mediaRecorder.start();
          startButton.disabled = true;
          stopButton.disabled = false;
        });

        stopButton.addEventListener("click", () => {
          mediaRecorder.stop();
          startButton.disabled = false;
          stopButton.disabled = true;
        });
      });
    </script>
  </body>
</html>


Overwriting static/index.html


In [ ]:
%%writefile app01.py

from flask import Flask, request, jsonify, send_from_directory
import os

app = Flask(__name__, static_folder="static")  

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app01.py


ふむ．とりあえず音声ファイル.webmはアップロードできるようになった．
ただspeech_recgnitionではwebmは受け入れないので，wavファイルに変える必要がある

## その２　アップロードした音声ファイルをWavファイルに変換

In [ ]:
%%writefile app02.py

from flask import Flask, request, jsonify, render_template
import os
import subprocess

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")  # フロントエンドのHTMLを表示

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    convert_webm_to_wav(audio_path, "uploads/output.wav")
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})


def convert_webm_to_wav(input_path, output_path):
    command = [
        "ffmpeg",
        "-i", input_path,  # 入力ファイル
        "-ar", "16000",  # サンプリングレート 16kHz
        "-ac", "1",  # モノラル変換
        "-preset", "ultrafast",  # 速度最優先
        output_path
    ]
    subprocess.run(command, check=True)

# 使い方

if __name__ == "__main__":
    app.run(debug=True)


Writing app02.py


変換に少々時間取られるな．．．

## その３　アップロードを直接Wavファイルにする．

変換に少々時間がかかるのが気になるので，アップロードの段階で直接Wavファイルをアップできないか探ってみたら，Record.jsなるものがあるようだ．
https://github.com/mattdiamond/Recorderjs
Recorder.jsをダウンロードして
これをhtmlに組み込んでみる．

けど，最初やってみたら，recorder.jsでエラーでた．
CDNがあるようなので，そちらでやったらうまく行った．

In [52]:
%%writefile static/index.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>
    <!-- <script src="recorder.js"></script> --> 
    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;
                

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                    document.getElementById("aiResponse").textContent =
                    data.ai_response || "AIの応答なし。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index.html


app01.pyで実行．
CORSの問題でアップロードで弾かれているようだ．．．

flask_corsを使って，サーバの側でCORS問題を無視するように設定する．

In [ ]:
!pip install flask_cors

In [40]:
%%writefile app03.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app03.py


よし，とりあえず問題は解決した．
ハマった理由は，fetchのインタフェースをフォルダ名と勘違いしていたこと．つまり，app.routeでは/uploadとしているのに，javascriptの方で/uploadsとしていた．これにより当然ながらインタフェースがないわけで４０４エラーが返されれるということになっていた．分れば馬鹿馬鹿しい勘違いやった😂

あと，デバッグ環境ではルートディレクトリがprojectになるというところもハマった😂

## その４　Speech Recognitionにかける
よし，ここからはpythonの側の処理に集中

In [54]:
%%writefile app04.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)
            
        ai_response = "test"
        return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app04.py


ふむ．これでとりあえず，音声認識結果を返せるようになった．

## その５　 openai の　Chat＿Compelationを使う

In [55]:
%%writefile app05.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

        # AIの応答
        client = OpenAI()
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": text},
            ]
        )
        ai_response = completion.choices[0].message.content
        return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Writing app05.py


ふむ．とりあえずは単発会話はできるようになった．
現時点の違和感・修正したい点は
現状だと，入力テキストとレスポンスが同時に帰ってきてしまう．
どうにか，その部分をいじれないか．レスポンスをまたつに先に入力テキストを返して，画面に表示させておいて，レスポンスが帰ってきたら，改めてそれを返すという感じ．

## その6　入力テキストとレスポンスを分けて表記できるようにする
やるとしたら，postを2回に分ける形かな？
Copilotに聞いてみたらWeb Socketを使えばできるとな．．．
とりあえずやってみるか．

In [ ]:
!pip install flask-socketio

In [5]:
%%writefile app06.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)
socketio = SocketIO(app)

@app.route("/")
def index():
    return send_from_directory("static", "index06.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

def get_ai_response(text):
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": text},
        ]
    )
    ai_response = completion.choices[0].message.content
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response})

if __name__ == "__main__":
    socketio.run(app, debug=True)

Overwriting app06.py


In [7]:
%%writefile static/index06.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                document.getElementById("aiResponse").textContent = data.ai_response;
            });
        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index06.html


ふむ．入力とレスポンスを別々に表記できるようになった．

## その７　継続的な会話を出来るようにする．
とりあえず，フロントエンドでログを記載する部分については別に考えて，まずはパイソンで動かす

In [8]:
%%writefile app07.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

@app.route("/")
def index():
    return send_from_directory("static", "index06.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response})

if __name__ == "__main__":
    socketio.run(app, debug=True)

Writing app07.py


ふむ．継続性のある会話もできるようになった．
じゃあ次は，TTSを組み込みたいね．

調べたらVoice VoxであればAPIが使えるとのこと．ただ，このAPIはあくまでローカルサーバで動くものになっている．
ローカルでどれくらい動作時間かかるんやろ？？
とりあえず試すか・・・．

## その8　音声合成機能を使ってみる
VoiceVox GUIを立ち上げておく．これによりローカルにVoiceVoxが立ち上がりAPIが使える．
そのまま関数を作ってくれている人がいたので拝借

https://zenn.dev/zenn24yykiitos/articles/fff3c954ddf42c

In [9]:
!pip install requests


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%%writefile app08.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    return send_from_directory("static", "index08.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    return send_file(os.path.join("uploads",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})

    # 処理時間の計算
    ai_time = time.time() - start
    print(f"処理時間: {ai_time} [sec]") 

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    print(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1, filename="uploads/output.wav"):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        with open(filename, 'wb') as f:
            f.write(synthesis_response.content)
        print(f"音声が {filename} に保存されました。")
        return "output.wav"
    else:
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    socketio.run(app, debug=True)

Writing app08.py


In [1]:
%%writefile static/index08.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                document.getElementById("aiResponse").textContent = data.ai_response;
                
                // 音声ファイルを自動再生する処理
                if (data.audio) {
                    const audio = new Audio(`/audio/${data.audio}`);
                    audio.play();
                }    
            });
        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index08.html


とりあえず動く形にはできた！！
ただ，どうしてもレスポンスは遅い．．．．

## その９：loggingモジュールをつかってみることにする

In [8]:
%%writefile voicecahtapp09.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------
#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log"
)
#--------------------------------------------------


# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index08.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.info("音声ファイルをアップロードします。")
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav")
    logging.debug(f"Saving audio file to {audio_path}")
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            logging.debug(text)
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    return send_file(os.path.join("output",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})

    # 処理時間の計算
    ai_time = time.time() - start
    logging.debug(f"処理時間: {ai_time} [sec]")
    print(f"AIレスポンス時間: {ai_time} [sec]") 

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    logging.debug(f"音声合成時間: {voice_time} [sec]")
    print(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        filename = f"output_{len(messages)}.wav"
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.debug(f"音声が {filename} に保存されました。")
        print(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    socketio.run(app, debug=True)

Overwriting voicecahtapp09.py


## その１０：htmlをチャットログが残るようにする．

In [2]:
%%writefile voicecahtapp10.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------
#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)
#--------------------------------------------------


# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index10.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.info("音声ファイルをアップロードします。")
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav")
    audio_file.save(audio_path)
    logging.info(f"Saved audio file to {audio_path}")

    # 音声認識
    r = sr.Recognizer()
    start_time = time.time()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    logging.info(f"音声認識結果: {text}")
    logging.info(f"音声認識時間: {time.time() - start_time} [sec]")

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    logging.info(f"音声ファイル {filename} を返します。")
    return send_file(os.path.join("output",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")

    # 処理時間の計算
    ai_time = time.time() - start
    logging.info(f"AIレスポンス時間: {ai_time} [sec]")

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    logging.info(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        filename = f"output_{len(messages)}.wav"
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.info(f"音声が {filename} に保存されました。")
        print(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)

Overwriting voicecahtapp10.py


In [5]:
%%writefile static/index10.html

<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css">

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <div id="chatlog"></div>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                const markdownText = data.ai_response;
                const htmlContent = marked.parse(markdownText);
                document.getElementById("chatlog").innerHTML += `<div class="assistant">${htmlContent}</div>`;
                
                // 音声ファイルを自動再生する処理
                if (data.audio) {
                    const audio = new Audio(`/audio/${data.audio}`);
                    audio.play();
                }
                document.getElementById("startRecording").disabled = false;
                document.getElementById("stopRecording").disabled = true;                    
            });

            // Spaceキーが押されたときにstartRecordingボタンをクリック
            document.addEventListener("keydown", (event) => {
                if(document.getElementById("startRecording").disabled){ 
                    console.log("処理中のため入力はできません");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    document.getElementById("startRecording").click();
                }
            });

            // Spaceキーから指が離されたときにstopRecordingボタンをクリック
            document.addEventListener("keyup", (event) => {
                if(document.getElementById("stopRecording").disabled){
                    console.log("不正な録音停止操作です");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    document.getElementById("stopRecording").click();
                }
            });

        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    if(data.text){
                         document.getElementById("chatlog").innerHTML += `<div class="user">${marked.parse(data.text)}</div>`;
                    }
                    else console.log("Error: 音声を認識できませんでした。");
                })
                .catch((error) => {
                    console.error("Upload failed:");
                });
            });

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = true;
        });


    </script>
</body>
</html>


Overwriting static/index10.html


概ね上手く行くようにできた．
後の問題は，AIからのレスポンスがマークダウン形式になっているのをうまく表記することかな．

-> 対応した　marked.jsなんてのがあるんや．これ使えば簡単！

## その１１　キャラクターボイスの選択

https://zenn.dev/zenn24yykiitos/articles/f3e983fe650e08

これを参考に，フロントエンドからキャラクターを選べるようにする．

In [8]:
%%writefile voicechatapp11.py

from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index11.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.info("音声ファイルをアップロードします。")
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    #audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)
    logging.info(f"Saved audio file to {audio_path}")

    # 音声認識
    r = sr.Recognizer()
    start_time = time.time()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    logging.info(f"音声認識結果: {text}")
    logging.info(f"音声認識時間: {time.time() - start_time} [sec]")

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    speaker = request.form["speaker"]
    logging.info(f"AIの応答を取得します。音声合成スピーカーID: {speaker}")
    threading.Thread(target=get_ai_response, args=(text, speaker)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    logging.info(f"音声ファイル {filename} を返します。")
    return send_file(os.path.join("output",filename))

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = "http://localhost:50021/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    filename = synthesize_voice(text, speaker)
    return jsonify({"audio": filename})


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# AIの応答を取得する関数 
def get_ai_response(text, speaker):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")

    # 処理時間の計算
    ai_time = time.time() - start
    logging.info(f"AIレスポンス時間: {ai_time} [sec]")

    # 音声合成
    filename = synthesize_voice(ai_response, speaker)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    logging.info(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        #filename = f"output_{len(messages)}.wav" #合成音声を全部残すならこっちをON
        filename = "output.wav" #合成音声を全部残さないならこっちをON
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.info(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


Overwriting voicechatapp11.py


In [7]:
%%writefile static/index11.html
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
  </head>

  <body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <select id="speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="chatlog"></div>

    <script>
      navigator.mediaDevices
        .getUserMedia({ audio: true })
        .then((stream) => {
          window.stream = stream;
        })
        .catch((error) => {
          console.error("Error accessing the microphone: " + error);
        });
      let audioContext;
      let recorder;
      let audioBlob;

      document.addEventListener("DOMContentLoaded", () => {
        const socket = io();

        // SpeakerIDリストを取得
        fetch("/speaker_ids")
          .then((response) => response.json())
          .then((data) => {
            const speakerSelect = document.getElementById("speakerSelect");
            speakerSelect.innerHTML = data.join("");
          });

        // AIの応答を受信したときの処理
        socket.on("ai_response", (data) => {
          const markdownText = data.ai_response;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "chatlog"
          ).innerHTML += `<div class="assistant">${htmlContent}</div>`;

          // 音声ファイルを自動再生する処理
          console.log("audioリクエスト");
          const audio = new Audio(`/audio/${data.audio}`);
          console.log("audio受信しました", audio);
          audio.play();

          document.getElementById("startRecording").disabled = false;
          document.getElementById("stopRecording").disabled = true;
          document.getElementById("selectSpeaker").disabled = false;
        });

        // Spaceキーが押されたときにstartRecordingボタンをクリック
        document.addEventListener("keydown", (event) => {
          if (document.getElementById("startRecording").disabled) {
            console.log("処理中のため入力はできません");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            document.getElementById("startRecording").click();
          }
        });

        // Spaceキーから指が離されたときにstopRecordingボタンをクリック
        document.addEventListener("keyup", (event) => {
          if (document.getElementById("stopRecording").disabled) {
            console.log("不正な録音停止操作です");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            document.getElementById("stopRecording").click();
          }
        });

        document.getElementById("speakerTest").addEventListener("click", () => {
          const speaker = document.getElementById("speakerSelect").value;
          fetch("/speaker_test", {
            method: "POST",
            headers: {
              "Content-Type": "application/json",
            },
            body: JSON.stringify({ speaker }),
          })
            .then((response) => response.json())
            .then((data) => {
              const audio = new Audio(`/audio/${data.audio}`);
              audio.play();
            });
        });

        document
          .getElementById("startRecording")
          .addEventListener("click", () => {
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(window.stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
            document.getElementById("selectSpeaker").disabled = true;
          });

        document
          .getElementById("stopRecording")
          .addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
              audioBlob = blob;

              if (!audioBlob) {
                console.error("No audio to upload");
                return;
              }

              const formData = new FormData();
              formData.append("file", audioBlob, "recorded_audio.wav");

              const speaker = document.getElementById("speakerSelect").value;
              formData.append("speaker", speaker);

              fetch("/upload", {
                method: "POST",
                body: formData,
              })
                .then((response) => response.json())
                .then((data) => {
                  if (data.text) {
                    document.getElementById(
                      "chatlog"
                    ).innerHTML += `<div class="user">${marked.parse(
                      data.text
                    )}</div>`;
                  } else console.log("Error: 音声を認識できませんでした。");
                })
                .catch((error) => {
                  console.error("Upload failed:");
                });
            });

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = true;
          });
      });
    </script>
  </body>
</html>

Overwriting static/index11.html


## その１２　よりリアルタイム化したい
テキストを句点や感嘆符などの句単位に区切り，それをflask側でストリーミングで音声合成し，フロントエンドにストリーミで返す．フロントエンドでは受け取ったストリームを再生する．

In [ ]:
from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging
import re

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index12.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = "http://localhost:50021/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    filename = synthesize_voice(text, speaker)
    return jsonify({"audio": filename})


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) #, 'audio': filename})
    else:
        return jsonify({"error": "Failed to get AI response"}), 400

    # AIの応答から音声合成
    speaker = request.form["speaker"]
    filename = synthesize_voice(ai_response, speaker)
    ## WebSocketを通じてクライアントに通知
    if filename:
        socketio.emit('play_audio', {'audio': filename})
    else:
        return jsonify({"error": "Failed to synthesize voice"}), 400
    
    return jsonify({"info": "Process Succeeded"}), 200


# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    return send_file(os.path.join("output",filename))



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    # 音声認識の結果を最初に返す
    socketio.emit("SpeechRecognition",{"text": text})
    
    # 続けてストリームで音声合成
    ## まずは openai で応答を取得
    ai_response = get_ai_response(text)
    socketio.emit("AIResponse", {"ai_response": ai_response})

    ## 音声合成
    speaker = request.form["speaker"]
    logging.info(f"AIの応答を取得します。音声合成スピーカーID: {speaker}")    
    

    def generate():
        yield from synthesize_streaming(ai_response, speaker)

    socketio.emit("Streaming", {
        "Response": Response(
            stream_with_context(generate()),
            content_type="application/octet-stream"
        )
    })
    
    return jsonify({"info": "Process Succeeded"}), 200





#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response


# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        filename = f"output_{len(messages)}.wav" #合成音声を全部残すならこっちをON
        #filename = "output.wav" #合成音声を全部残さないならこっちをON
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.info(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# テキストを句単位に区切る
def preprocess_text(text):
    # テキストの前処理
    text = re.sub(r"[。．.]", "。\n", text)
    text = re.sub(r"[？?]", "？\n", text)
    text = re.sub(r"[！!]", "！\n", text)
    return text

# テキストを句ごとに音声合成してストリーミング
def synthesize_streaming(text, speaker):
    # テキストを句単位に区切る
    logging.debug("テキストを句単位に区切ります。")
    text = preprocess_text(text)
    sentences = text.split("\n")

    # 句ごとに音声合成
    for sentence in sentences:
        if sentence == "": continue
        
        ## クエリ
        query_response = requests.post(
            f'http://localhost:50021/audio_query', 
            params={'text': sentence, 'speaker': speaker}
        )
        if query_response.status_code != 200:
            logging.error(f"Error in audio_query: {query_response.text}")
            return
        
        ## 音声合成
        logging.debug("音声データを生成します。")
        with requests.post(
            f'http://localhost:50021/synthesis', 
            params={'speaker': speaker}, 
            json=query_response.json(), 
            stream=True
        ) as synthesis_response:
            if synthesis_response.status_code != 200:
                logging.error(f"Error in synthesis: {synthesis_response.text}")
                return
            yield "---start---\n".encode("utf-8")
            for chunk in synthesis_response.iter_content(chunk_size=1024):
                logging.info("チャンク生成")
                yield chunk
            yield "---end---\n".encode("utf-8")

            time.sleep(0.2)
        
    


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


Writing voicechatapp12.py


In [ ]:
%%writefile static/index12.html
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
  </head>

  <body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <button id="stopRecordingWithStreaming">停止とストリーミング処理</button>
    <select id="speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="chatlog"></div>

    <script>
      navigator.mediaDevices
        .getUserMedia({ audio: true })
        .then((stream) => {
          window.stream = stream;
        })
        .catch((error) => {
          console.error("Error accessing the microphone: " + error);
        });
      let audioContext;
      let recorder;
      let audioBlob;

      // フォーム要素取得
      const startRecButton = document.getElementById("startRecording");
      const stopRecButton = document.getElementById("stopRecording");
      const stopRecwithStreamingButton = document.getElementById(
        "stopRecordingWithStreaming"
      );
      const speakerSelect = document.getElementById("speakerSelect");
      const speakerTestButton = document.getElementById("speakerTest");

      // 録音開始時のボタンを無効化
      function setBtnonStart() {
        startRecButton.disabled = true;
        stopRecButton.disabled = false;
        stopRecwithStreamingButton.disabled = false;
        speakerSelect.disabled = true;
        speakerTestButton.disabled = true;
      }

      // 処理中のボタン無効化
      function setBtnunderProcessing() {
        startRecButton.disabled = true;
        stopRecButton.disabled = true;
        stopRecwithStreamingButton.disabled = true;
        speakerSelect.disabled = true;
        speakerTestButton.disabled = true;
      }

      // 復帰時のボタン有効化
      function setBtnonRestart() {
        startRecButton.disabled = false;
        stopRecButton.disabled = true;
        stopRecwithStreamingButton.disabled = true;
        speakerSelect.disabled = false;
        speakerTestButton.disabled = false;
      }

      // 音声合成のストリーミング処理
      async function playSnetence(chunks, audioContext) {
        const combined = new Uint8Array(
          chunks.reduce((acc, chunk) => [...acc, ...chunk], [])
        );
        const audioBuffer = await audioContext.decodeAudioData(combined.buffer);
        const source = audioContext.createBufferSource();
        source.buffer = audioBuffer;
        source.connect(audioContext.destination);
        source.start();

        await new Promise((resolve) => {
          source.onended = resolve;
        });
      }

      document.addEventListener("DOMContentLoaded", () => {
        const socket = io();

        // SpeakerIDリストを取得
        fetch("/speaker_ids")
          .then((response) => response.json())
          .then((data) => {
            speakerSelect.innerHTML = data.join("");
          });

        // 音声認識の結果を受信
        socket.on("SpeechRecognition", (data) => {
          const markdownText = data.text;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "chatlog"
          ).innerHTML += `<div class="user">${htmlContent}</div>`;
        });

        // AIの応答を受信したときの処理
        socket.on("ai_response", (data) => {
          const markdownText = data.ai_response;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "chatlog"
          ).innerHTML += `<div class="assistant">${htmlContent}</div>`;
        });

        // 音声ファイルを再生する処理
        socket.on("play_audio", (data) => {
          const audio = new Audio(`/audio/${data.audio}`);
          audio.play();
        });

        //Stremingで音声合成の結果を受信
        socket.on("Streaming", async (data) => {
          const reader = data.Response.body.getReader();
          const audioContext = new AudioContext();
          let chunks = [];
          let proccesing = false;

          while (true) {
            const { done, value } = await reader.read();
            if (done) break;
            const textChunk = new TextDecorder("utf-8").decode(value);
            if (textChunk.include("---start---")) {
              if (chunks.length > 0 && !proccesing) {
                proccesing = true;
                await playSnetence(chunks, audioContext);
                chunks = [];
                proccesing = false;
              }
            } else if (textChunk.include("---end---")) {
              proccesing = true;
              await playSnetence(chunks, audioContext);
              chunks = [];
              proccesing = false;
            } else {
              chunks.push(value);
            }
          }
        });

        // Spaceキーが押されたときにstartRecordingボタンをクリック
        document.addEventListener("keydown", (event) => {
          if (startRecButton.disabled) {
            console.log("処理中のため入力はできません");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            startRecButton.click();
          }
        });

        // Spaceキーから指が離されたときにstopRecordingボタンをクリック
        document.addEventListener("keyup", (event) => {
          if (stopRecButton.disabled) {
            console.log("不正な録音停止操作です");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            stopRecButton.click();
          }
        });

        //Speakerの音声確認テスト
        speakerTestButton.addEventListener("click", () => {
          const speaker = speakerSelect.value;
          fetch("/speaker_test", {
            method: "POST",
            headers: {
              "Content-Type": "application/json",
            },
            body: JSON.stringify({ speaker }),
          })
            .then((response) => response.json())
            .then((data) => {
              const audio = new Audio(`/audio/${data.audio}`);
              audio.play();
            });
        });

        // 録音開始ボタンがクリックされたときの処理
        startRecButton.addEventListener("click", () => {
          audioContext = new AudioContext();
          const source = audioContext.createMediaStreamSource(window.stream);
          recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
          recorder.record();

          // ボタンを無効化
          setBtnonStart();
        });

        // 録音停止ボタンがクリックされたときの処理
        stopRecButton.addEventListener("click", () => {
          // ボタンを無効化
          setBtnunderProcessing();

          // 録音を停止
          recorder.stop();

          // 録音した音声をファイルに保存して送信
          recorder.exportWAV((blob) => {
            audioBlob = blob;

            if (!audioBlob) {
              console.error("No audio to upload");
              return;
            }

            const formData = new FormData();
            formData.append("file", audioBlob, "recorded_audio.wav");

            const speaker = speakerSelect.value;
            formData.append("speaker", speaker);

            fetch("/upload", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                console.log(data);
                // ボタン状態の初期化
                setBtnonRestart();
              })
              .catch((error) => {
                console.error("Upload failed:");
                // ボタン状態の初期化
                setBtnonRestart();
              });
          });
        });

        stopRecwithStreamingButton.addEventListener("click", () => {
          // ボタンを無効化
          setBtnunderProcessing();

          // 録音を停止
          recorder.stop();

          // 録音した音声をファイルに保存して送信
          recorder.exportWAV((blob) => {
            audioBlob = blob;

            if (!audioBlob) {
              console.error("No audio to upload");
              return;
            }

            const formData = new FormData();
            formData.append("file", audioBlob, "recorded_audio.wav");

            const speaker = speakerSelect.value;
            formData.append("speaker", speaker);

            fetch("/streaming", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                console.log(data);
                // ボタン状態の初期化
                setBtnonRestart();
              })
              .catch((error) => {
                console.error("Upload failed:");
                // ボタン状態の初期化
                setBtnonRestart();
              });
          });
        });

        // ボタン状態の初期化
        setBtnonRestart();
      });
    </script>
  </body>
</html>


Writing static/index12.html


まだストリーム再生はできていない．
けど，ソケット通信でPush型で動くようにした．
プログラム的にはだいぶ綺麗にはなったとは思う．

## その１３　改めて音声をストリームで受け取る形にする
GPTの出力を文ごとに切り分けて，1文ずつVoice Voxに送り，合成された音声を順次mp３でクライアントに送る．
クライアントでは受け取った音声を順次キューに入れていく．

In [ ]:
%%writefile voicechatapp13.py

from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import re
from pydub import AudioSegment
from io import BytesIO

#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index13.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    synthesize_response = synthesize_voice(ai_response, speaker)

    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) #, 'audio': filename})
    else:
        return jsonify({"error": "Failed to get AI response"}), 400

    # AIの応答から音声合成
    speaker = request.form["speaker"]
    synthesize_response = synthesize_voice(ai_response, speaker)


    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) #, 'audio': filename})
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    

    ## 音声合成
    speaker = request.form["speaker"]

    # テキストを句単位に区切る
    logging.debug("テキストを句単位に区切ります。")
    text = preprocess_text(ai_response)
    sentences = text.split("\n")

    # 句ごとに音声合成
    for sentence in sentences:
        if sentence == "": continue
        
        ## 音声合成
        synthesize_response=synthesize_voice(sentence, speaker)
        if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400

        ## 合成した音声をmp3化
        audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
        mp3_data  = BytesIO()
        audio.export(mp3_data , format="mp3")
        mp3_data .seek(0)

        ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
        socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
        time.sleep(0.5)

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response


# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# テキストを句単位に区切る
def preprocess_text(text):
    # テキストの前処理
    text = re.sub(r"[。．.]", "。\n", text)
    text = re.sub(r"[？?]", "？\n", text)
    text = re.sub(r"[！!]", "！\n", text)
    return text


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


In [ ]:
%%witefile /static/index13.html
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
  </head>

  <body>
    <h1>WAV録音アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <input type="radio" id="streaming" name="Method" value="/upload" checked>まとめて再生(基本)</radio>
    <input type="radio" id="streaming" name="Method" value="/streaming" >ストリーミング</radio>
    <select id="speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="chatlog"></div>

    <script>
      navigator.mediaDevices
        .getUserMedia({ audio: true })
        .then((stream) => {
          window.stream = stream;
        })
        .catch((error) => {
          console.error("Error accessing the microphone: " + error);
        });

      let audioContext;
      let recorder;
      let audioBlob;
      let audioQueue = [];
      let isPlaying = false;

      // フォーム要素取得
      const startRecButton = document.getElementById("startRecording");
      const stopRecButton = document.getElementById("stopRecording");
      const speakerSelect = document.getElementById("speakerSelect");
      const speakerTestButton = document.getElementById("speakerTest");

      // 録音開始時のボタンを無効化
      function setBtnonStart() {
        startRecButton.disabled = true;
        stopRecButton.disabled = false;
        speakerSelect.disabled = true;
        speakerTestButton.disabled = true;
      }

      // 処理中のボタン無効化
      function setBtnunderProcessing() {
        startRecButton.disabled = true;
        stopRecButton.disabled = true;
        speakerSelect.disabled = true;
        speakerTestButton.disabled = true;
      }

      // 復帰時のボタン有効化
      function setBtnonRestart() {
        startRecButton.disabled = false;
        stopRecButton.disabled = true;
        speakerSelect.disabled = false;
        speakerTestButton.disabled = false;
      }


      document.addEventListener("DOMContentLoaded", () => {
        const socket = io();

        // SpeakerIDリストを取得
        fetch("/speaker_ids")
          .then((response) => response.json())
          .then((data) => {
            speakerSelect.innerHTML = data.join("");
          });

        // 音声認識の結果を受信
        socket.on("SpeechRecognition", (data) => {
          const markdownText = data.text;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "chatlog"
          ).innerHTML += `<div class="user">${htmlContent}</div>`;
        });

        // AIの応答を受信したときの処理
        socket.on("ai_response", (data) => {
          const markdownText = data.ai_response;
          const htmlContent = marked.parse(markdownText);
          document.getElementById(
            "chatlog"
          ).innerHTML += `<div class="assistant">${htmlContent}</div>`;
        });

        // 音声ファイルを再生する処理
        socket.on("play_audio", async(data) => {
            const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
            const audioUrl = URL.createObjectURL(audioBlob);

            // キューに登録
            audioQueue.push(audioUrl);

            // 再生中でなければ再生
            if (!isPlaying) {
                playAudio();
            }
            // const audio = new Audio(audioUrl);
            // audio.play();
        });

        // Queueに登録された音声ファイルを再生する処理
        async function playAudio() {
            // 再生する音声ファイルがなければ終了
            if (audioQueue.length === 0) {
                isPlaying = false;
                return;
            }

            isPlaying = true;
            const audioUrl = audioQueue.shift();
            const audio = new Audio(audioUrl);
            audio.play();

            // 再生が終了したら次の音声ファイルを再生
            audio.onended = () => {
                playAudio();
            };
        }

        // //Stremingで音声合成の結果を受信
        // socket.on("Streaming", async (data) => {

        // });

        // Spaceキーが押されたときにstartRecordingボタンをクリック
        document.addEventListener("keydown", (event) => {
          if (startRecButton.disabled) {
            console.log("処理中のため入力はできません");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            startRecButton.click();
          }
        });

        // Spaceキーから指が離されたときにstopRecordingボタンをクリック
        document.addEventListener("keyup", (event) => {
          if (stopRecButton.disabled) {
            console.log("不正な録音停止操作です");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            stopRecButton.click();
          }
        });

        //Speakerの音声確認テスト
        speakerTestButton.addEventListener("click", () => {
          const speaker = speakerSelect.value;
          fetch("/speaker_test", {
            method: "POST",
            headers: {
              "Content-Type": "application/json",
            },
            body: JSON.stringify({ speaker }),
          })
            .then((response) => response.json())
            .then((data) => {
              console.log(data);
            });
        });

        // 録音開始ボタンがクリックされたときの処理
        startRecButton.addEventListener("click", () => {
          audioContext = new AudioContext();
          const source = audioContext.createMediaStreamSource(window.stream);
          recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
          recorder.record();

          // ボタンを無効化
          setBtnonStart();
        });

        // 録音停止ボタンがクリックされたときの処理
        stopRecButton.addEventListener("click", () => {
          // ボタンを無効化
          setBtnunderProcessing();

          // 録音を停止
          recorder.stop();

          // 録音した音声をファイルに保存して送信
          recorder.exportWAV((blob) => {
            audioBlob = blob;
            if (!audioBlob) {
              console.error("No audio to upload");
              return;
            }

            const formData = new FormData();
            formData.append("file", audioBlob, "recorded_audio.wav");

            const speaker = speakerSelect.value;
            formData.append("speaker", speaker);

            const method = document.querySelector('input[name="Method"]:checked').value;

            fetch(method, {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                console.log(data);
                // ボタン状態の初期化
                setBtnonRestart();
              })
              .catch((error) => {
                console.error("Upload failed:");
                // ボタン状態の初期化
                setBtnonRestart();
              });
          });
        });

        // ボタン状態の初期化
        setBtnonRestart();
      });

      // ページを離れるときにストリームを停止
      window.addEventListener("beforeunload", () => {
        if (window.stream) {
          window.stream.getTracks().forEach((track) => {
            track.stop();
          });
        }
      });
    </script>
  </body>
</html>


だいぶ苦労したけど，少しずつ形になってきた．
このあとはGPTからの返答もストリーミングで受け取れるようにする．

## その１４　GPTからの返答をストリームで受け取る．
これの場合，GPTからのストリーム出力を一旦キープして文末に来たところで一気に処理をかけるという処理になる．

In [ ]:
%%writefile voicechatapp14.py

from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import re
from pydub import AudioSegment
from io import BytesIO

#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index14.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    synthesize_response = synthesize_voice(ai_response, speaker)

    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # # AIの応答をストリームで生成
    # socketio.emit('ai_response', {'ai_response': "---Start---"}) # 開始を通知
    # for ai_response in generate_ai_response(text):
    #     ## WebSocketを通じてクライアントに通知
    #     if ai_response:
    #         socketio.emit('ai_response', {'ai_response': ai_response}) 
    #     else:
    #         return jsonify({"error": "Failed to get AI response"}), 400
    # socketio.emit('ai_response', {'ai_response': "---End---"}) # 終了を通知

    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) 
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    
    # AIの応答から音声合成
    speaker = request.form["speaker"]
    synthesize_response = synthesize_voice(ai_response, speaker)


    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})

    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    

    # AIの応答を句単位でストリームするとともに．句単位で音声合成もしていく
    """
    現状では，多分句の表示と音声が同期しない．句は順に表示されていくけど，音声はキューに入って順に再生されるので．
    これを同期させようと思うと，Javascriptに句を送ったものも一旦キューに入れて，音声と句を同時に処理するようにしないといけな
    い．
    できなくはないか・・・
    """
    speaker = request.form["speaker"]
    socketio.emit('ai_stream', {'ai_stream': "---Start---"}) # 開始を通知
    for sentence in generate_ai_response(text):
        ## WebSocketを通じてクライアントに通知
        if sentence:
            # 音声合成
            synthesize_response=synthesize_voice(sentence, speaker)
            if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
            ## 合成した音声をmp3化
            audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
            mp3_data  = BytesIO()
            audio.export(mp3_data , format="mp3")
            mp3_data .seek(0)
            ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
            socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
            socketio.emit('ai_stream', {'ai_stream': sentence})
            ## 0.5秒の無音を入れる．これで句の切り分けが聞きやすくなると思う．
            silent_audio = AudioSegment.silent(duration=500)
            mp3_data  = BytesIO()
            silent_audio.export(mp3_data , format="mp3")
            mp3_data .seek(0)
            socketio.emit('play_audio', {'audio': mp3_data.getvalue()}) 
        else:
            return jsonify({"error": "Failed to get AI response"}), 400
    socketio.emit('ai_stream', {'ai_stream': "---End---"}) # 終了を通知

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response

# OpenAIのAPIを呼び出してAIの応答をストリームで生成する関数
def generate_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream = True
    )

    sentens = "" # 句を構成するためのバッファ
    message = "" # プロンプトに含めるためにチャンクを結合させるためのためのバッファ
    for chunk in completion:
        # きちんとしたチャンクが帰ってきているかのチェック
        if "choices" in chunk.to_dict() and len(chunk.choices) > 0: #to_dict：辞書型に変えないと”choices”が見つからないようなので
            content  = chunk.choices[0].delta.content
            if content:
                message += content
                # 1文字ずつ取り出してチェックする
                for i in range(len(content)):
                    char = content[i]
                    sentens += char
                    if char in "。．.？?！!\n": #今見ているのが区切り文字だった場合
                        if i < len(content)-1: # i が最後の文字でないなら，次の文字をチェック
                            if content[i+1] not in "。．.？?！!\n": #次の文字が区切り文字でないならyield
                                logging.debug(f"句: {sentens}")
                                yield sentens
                                sentens = ""
                            else: #もし次の文字が区切り文字なら，現時点の区切り文字はスルー
                                continue
                        else: #iが最後の文字の場合，現時点でyield
                            logging.debug(f"句: {sentens}")
                            yield sentens
                            sentens = ""
    # 最後の句を返す
    if sentens:
        yield sentens
    
    # message をmessagesに追加
    messages.append({"role": "assistant", "content": message})
    logging.info(f"AIの応答: {message}")



# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# テキストを句単位に区切る
def preprocess_text(text):
    # テキストの前処理
    text = re.sub(r"[。．.]", "。\n", text)
    text = re.sub(r"[？?]", "？\n", text)
    text = re.sub(r"[！!]", "！\n", text)
    return text


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)


Writing voicechatapp14.py


In [ ]:
%%writefile /static/index14.html
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
  </head>

  <body>
    <h1>WAV録音アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <input type="radio" id="streaming" name="Method" value="/upload" checked>まとめて再生(基本)</radio>
    <input type="radio" id="streaming" name="Method" value="/streaming" >ストリーミング</radio>
    <select id="speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="chatlog"></div>

    <script>
      navigator.mediaDevices
        .getUserMedia({ audio: true })
        .then((stream) => {
          window.stream = stream;
        })
        .catch((error) => {
          console.error("Error accessing the microphone: " + error);
        });

      let audioContext;
      let recorder;
      let audioBlob;
      let audioQueue = [];
      let isPlaying = false;

      // html要素取得
      const startRecButton = document.getElementById("startRecording");
      const stopRecButton = document.getElementById("stopRecording");
      const speakerSelect = document.getElementById("speakerSelect");
      const speakerTestButton = document.getElementById("speakerTest");
      const chatlog = document.getElementById("chatlog");

      // 録音開始時のボタンを無効化
      function setBtnonStart() {
        startRecButton.disabled = true;
        stopRecButton.disabled = false;
        speakerSelect.disabled = true;
        speakerTestButton.disabled = true;
      }

      // 処理中のボタン無効化
      function setBtnunderProcessing() {
        startRecButton.disabled = true;
        stopRecButton.disabled = true;
        speakerSelect.disabled = true;
        speakerTestButton.disabled = true;
      }

      // 復帰時のボタン有効化
      function setBtnonRestart() {
        startRecButton.disabled = false;
        stopRecButton.disabled = true;
        speakerSelect.disabled = false;
        speakerTestButton.disabled = false;
      }


      document.addEventListener("DOMContentLoaded", () => {
        const socket = io();

        // SpeakerIDリストを取得
        fetch("/speaker_ids")
          .then((response) => response.json())
          .then((data) => {
            speakerSelect.innerHTML = data.join("");
          });

        // 音声認識の結果を受信
        socket.on("SpeechRecognition", (data) => {
          const markdownText = data.text;
          const htmlContent = marked.parse(markdownText);
          chatlog.innerHTML += `<div class="user">${htmlContent}</div>`;
        });

        // AIの応答を受信したときの処理
        socket.on("ai_response", (data) => {
          const markdownText = data.ai_response;
          const htmlContent = marked.parse(markdownText);
          chatlog.innerHTML += `<div class="assistant">${htmlContent}</div>`;
        });

        // AIの応答ストリームを受信したときの処理
        let currentDiv="";
        socket.on("ai_stream", (data) => {
          if (data.ai_stream.includes("---Start---")) { 
            // 最初はdivを作成
            chatlog.innerHTML += `<div class="assistant"></div>`;
            const assistantDivs = chatlog.getElementsByClassName("assistant");
            currentDiv = assistantDivs[assistantDivs.length - 1];//作ったdivを取得
            return;
          }
          else if (data.ai_stream.includes("---End---") ){ 
            // 終了時は改めて中身をマークダウンで書き直す．
            currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
            currentDiv = ""; //初期化
            return;
          }
          else{
            // 途中の場合はnakedなテキストを追加
            currentDiv.innerHTML += data.ai_stream;
          }
        });

        // 音声ファイルを再生する処理
        socket.on("play_audio", async(data) => {
            const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
            const audioUrl = URL.createObjectURL(audioBlob);

            // キューに登録
            audioQueue.push(audioUrl);

            // 再生中でなければ再生
            if (!isPlaying) {
                playAudio();
            }
        });

        // Queueに登録された音声ファイルを再生する処理
        async function playAudio() {
            // 再生する音声ファイルがなければ終了
            if (audioQueue.length === 0) {
                isPlaying = false;
                return;
            }

            isPlaying = true;
            const audioUrl = audioQueue.shift();
            const audio = new Audio(audioUrl);
            audio.play();

            // 再生が終了したら次の音声ファイルを再生
            audio.onended = () => {
                playAudio();
            };
        }


        // Spaceキーが押されたときにstartRecordingボタンをクリック
        document.addEventListener("keydown", (event) => {
          if (startRecButton.disabled) {
            console.log("処理中のため入力はできません");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            startRecButton.click();
          }
        });

        // Spaceキーから指が離されたときにstopRecordingボタンをクリック
        document.addEventListener("keyup", (event) => {
          if (stopRecButton.disabled) {
            console.log("不正な録音停止操作です");
            return;
          }
          if (event.code === "Space" && !event.repeat) {
            stopRecButton.click();
          }
        });

        //Speakerの音声確認テスト
        speakerTestButton.addEventListener("click", () => {
          const speaker = speakerSelect.value;
          fetch("/speaker_test", {
            method: "POST",
            headers: {
              "Content-Type": "application/json",
            },
            body: JSON.stringify({ speaker }),
          })
            .then((response) => response.json())
            .then((data) => {
              console.log(data);
            });
        });

        // 録音開始ボタンがクリックされたときの処理
        startRecButton.addEventListener("click", () => {
          audioContext = new AudioContext();
          const source = audioContext.createMediaStreamSource(window.stream);
          recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
          recorder.record();

          // ボタンを無効化
          setBtnonStart();
        });

        // 録音停止ボタンがクリックされたときの処理
        stopRecButton.addEventListener("click", () => {
          // ボタンを無効化
          setBtnunderProcessing();

          // 録音を停止
          recorder.stop();

          // 録音した音声をファイルに保存して送信
          recorder.exportWAV((blob) => {
            audioBlob = blob;
            if (!audioBlob) {
              console.error("No audio to upload");
              return;
            }

            const formData = new FormData();
            formData.append("file", audioBlob, "recorded_audio.wav");

            const speaker = speakerSelect.value;
            formData.append("speaker", speaker);

            const method = document.querySelector('input[name="Method"]:checked').value;

            fetch(method, {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                console.log(data);
                // ボタン状態の初期化
                setBtnonRestart();
              })
              .catch((error) => {
                console.error("Upload failed:");
                // ボタン状態の初期化
                setBtnonRestart();
              });
          });
        });

        // ボタン状態の初期化
        setBtnonRestart();
      });

      // ページを離れるときにストリームを停止
      window.addEventListener("beforeunload", () => {
        if (window.stream) {
          window.stream.getTracks().forEach((track) => {
            track.stop();
          });
        }
      });

    </script>
  </body>
</html>


なんとかできた．

Chunkの文章への再構築と，一方で文末かどうかの区切りがめちゃくちゃ面倒やった（苦笑）

現状では，音声生成のスピードがそこまでではないので，そこまでキュニー蓄積させるということもないが，もしもっと合成のスピードが上がれば，先に文章がバーっと表示されていく，ということが起こり得ると思う．
まあ，それも悪くはないが・・・


## その１５　音声とセンテンス表示の同期をとる．
チャレンジしてみるか

In [ ]:
%%writefile voicechatapp15.py

from flask import Flask, request, Response, jsonify, send_from_directory, send_file, stream_with_context
from flask_cors import CORS
from flask_socketio import SocketIO, emit
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import time
import logging
import re
from pydub import AudioSegment
from io import BytesIO

#　環境変数の読み込み
load_dotenv()

# VoiceVox APIのエンドポイント
VOICEVOX_API_URL = "http://localhost:50021"


# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)

#--------------------------------------------------
# Flaskのエンドポイントの作成
#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index15.html")

# VoiceVoxのSpeakerIDリストを取得するエンドポイント
@app.route("/speaker_ids")
def get_speaker_ids():
    url = f"{VOICEVOX_API_URL}/speakers"  # VOICEVOX APIのエンドポイント
    try:
        response = requests.get(url)
    except Exception as e:
        logging.error(f"Error: {e}")
        return jsonify([])

    voicevox_speakers = []
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker['name']
            style_names = [style['name'] for style in speaker['styles']]
            style_ids = [style['id'] for style in speaker['styles']]
            for style_id, style_name in zip(style_ids, style_names):
                voicevox_speakers.append(f"<option value={style_id}>Speaker: {name}, {style_name} </option>")
        logging.info("speaker_ids を取得しました。")
        return jsonify(voicevox_speakers)
    else:
        logging.error(f"Error: {response.status_code}")
        return jsonify([])    

# VoiceVoxの音声テストを行うエンドポイント
@app.route("/speaker_test" , methods=["POST"])
def speaker_test():
    speaker = request.json["speaker"]
    text = "こんにちは．初めまして．何かお手伝いできることはありますか？"
    synthesize_response = synthesize_voice(text, speaker)

    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})
    return jsonify({"info": "Speaker Test Process Succeeded"}), 200


# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav") #Uploadされたファイルを残すならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    
    # AIの応答を取得
    ai_response = get_ai_response(text)
    ## WebSocketを通じてクライアントに通知
    if ai_response:
        socketio.emit('ai_response', {'ai_response': ai_response}) 
    else:
        return jsonify({"error": "Failed to get AI response"}), 400
    
    # AIの応答から音声合成
    speaker = request.form["speaker"]
    synthesize_response = synthesize_voice(ai_response, speaker)


    # 合成した音声をmp3化
    if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
    audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
    mp3_data  = BytesIO()
    audio.export(mp3_data , format="mp3")
    mp3_data .seek(0)  

    # mp3データをWebSocketを通じてクライアントに通知
    socketio.emit('play_audio', {'audio': mp3_data.getvalue()})

    return jsonify({"info": "Uploard Process Succeeded"}), 200



# streaming処理するエンドポイント
@app.route("/streaming", methods=["POST"])
def streaming():
    # uploads ディレクトリがなければ作成
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    # uploads ディレクトリにファイルがあれば削除
    else:
        for file in os.listdir("uploads"):
            os.remove(os.path.join("uploads", file))

    # 音声ファイルをアップロード
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", "input.wav") #Uploadされたファイルを残さないならこっちをOn
    audio_file.save(audio_path)

    # 音声認識
    text = recognize_speech(audio_path)
    ## 音声認識の結果をWebSocketを通じてクライアントに通知
    if text:
        socketio.emit("SpeechRecognition",{"text": text})
    else:
        return jsonify({"error": "Failed to recognize speech"}), 400    
    

    # AIの応答を句単位でストリームするとともに．句単位で音声合成もしていく
    speaker = request.form["speaker"]
    socketio.emit('ai_stream', {'sentens': "---Start---"}) # 開始を通知
    for sentence in generate_ai_response(text):
        ## WebSocketを通じてクライアントに通知
        if sentence:
            # 音声合成
            synthesize_response=synthesize_voice(sentence, speaker)
            if synthesize_response is None: return jsonify({"error": "Failed to synthesize voice"}), 400
            ## 合成した音声をmp3化
            audio = AudioSegment.from_file(BytesIO(synthesize_response.content), format="wav")
            mp3_data  = BytesIO()
            audio.export(mp3_data , format="mp3")
            mp3_data .seek(0)
            ## mp3データをWebSocketを通じてクライアントに通知 ここでうまくキューに入れて連続再生させたい
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': sentence})
            # sentensの区切り文字が読点だったら，0.2秒の無音を入れる
            if sentence[-1] in ",，、":
                silent_audio = AudioSegment.silent(duration=10)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # sentensの区切り文字が読点でなかったら，0.5秒の無音を入れる
            else:
                silent_audio = AudioSegment.silent(duration=500)
                mp3_data  = BytesIO()
                silent_audio.export(mp3_data , format="mp3")
                mp3_data .seek(0)
            # 無音を送信
            socketio.emit('ai_stream', {'audio': mp3_data.getvalue(), 'sentens': "---silent---"})
        else:
            return jsonify({"error": "Failed to get AI response"}), 400
    socketio.emit('ai_stream', {'sentens': "---End---"}) # 終了を通知

    
    return jsonify({"info": "Process Succeeded"}), 200


#--------------------------------------------------
# Flaskの各エンドポイント内の処理関数
#--------------------------------------------------
# 音声認識を行う関数
def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    return text

# OpenAIのAPIを呼び出してAIの応答を取得する関数
def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")
    return ai_response

# OpenAIのAPIを呼び出してAIの応答をストリームで生成する関数
def generate_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream = True
    )

    sentens = "" # 句を構成するためのバッファ　
    message = "" # プロンプトに含めるためにチャンクを結合させるためのためのバッファ
    for chunk in completion:
        # きちんとしたチャンクが帰ってきているかのチェック
        if "choices" in chunk.to_dict() and len(chunk.choices) > 0: #to_dict：辞書型に変えないと”choices”が見つからないようなので
            content  = chunk.choices[0].delta.content
            if content:
                message += content
                # 1文字ずつ取り出してチェックする
                for i in range(len(content)):
                    char = content[i]
                    sentens += char
                    if char in ",，、。．.？?！!\n": #今見ているのが区切り文字だった場合（読点も区切りに含める）
                        if i < len(content)-1: # i が最後の文字でないなら，次の文字をチェック
                            if content[i+1] not in ",，、。．.？?！!\n": #次の文字が区切り文字でないならyield
                                logging.debug(f"句: {sentens}")
                                yield sentens
                                sentens = ""
                            else: #もし次の文字が区切り文字なら，現時点の区切り文字はスルー
                                continue
                        else: #iが最後の文字の場合，現時点でyield
                            logging.debug(f"句: {sentens}")
                            yield sentens
                            sentens = ""
    # 最後の句を返す
    if sentens:
        yield sentens
    
    # message をmessagesに追加
    messages.append({"role": "assistant", "content": message})
    logging.info(f"AIの応答: {message}")



# VoiceVox APIで音声合成を行なう関数
def synthesize_voice(text, speaker):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'{VOICEVOX_API_URL}/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'{VOICEVOX_API_URL}/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        logging.info("音声データを生成しました。")
        return synthesis_response
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        return None


# テキストを句単位に区切る
def preprocess_text(text):
    # テキストの前処理
    text = re.sub(r"[。．.]", "。\n", text)
    text = re.sub(r"[？?]", "？\n", text)
    text = re.sub(r"[！!]", "！\n", text)
    return text


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)




Writing voicechatapp15.py


In [ ]:
%%writefile static/index15.html
<html lang="ja">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>WAV録音アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css" />
</head>

<body>
    <h1>WAV録音アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording">録音停止</button>
    <input type="radio" id="streaming" name="Method" value="/upload" checked>まとめて再生(基本)</radio>
    <input type="radio" id="streaming" name="Method" value="/streaming">ストリーミング</radio>
    <select id="speakerSelect"></select>
    <button id="speakerTest">音声テスト</button>
    <div id="chatlog"></div>

    <script>
        navigator.mediaDevices
            .getUserMedia({ audio: true })
            .then((stream) => {
                window.stream = stream;
            })
            .catch((error) => {
                console.error("Error accessing the microphone: " + error);
            });

        // 音声処理用の変数
        let audioContext; // 音声処理用のコンテキスト
        let recorder;   // 録音用のオブジェクト
        let audioBlob;  // 録音した音声データ
        let audioQueue = [];    // 音声ファイルのキュー
        let sentensQueue = [];  // センテンスのキュー
        let isPlaying = false;  // 音声ファイル再生中かどうか
        let currentDiv = "";    // 現在のdiv要素


        // html要素取得
        const startRecButton = document.getElementById("startRecording");
        const stopRecButton = document.getElementById("stopRecording");
        const speakerSelect = document.getElementById("speakerSelect");
        const speakerTestButton = document.getElementById("speakerTest");
        const chatlog = document.getElementById("chatlog");

        // 録音開始時のボタンを無効化
        function setBtnonStart() {
            startRecButton.disabled = true;
            stopRecButton.disabled = false;
            speakerSelect.disabled = true;
            speakerTestButton.disabled = true;
        }

        // 処理中のボタン無効化
        function setBtnunderProcessing() {
            startRecButton.disabled = true;
            stopRecButton.disabled = true;
            speakerSelect.disabled = true;
            speakerTestButton.disabled = true;
        }

        // 復帰時のボタン有効化
        function setBtnonRestart() {
            startRecButton.disabled = false;
            stopRecButton.disabled = true;
            speakerSelect.disabled = false;
            speakerTestButton.disabled = false;
        }


        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            // SpeakerIDリストを取得
            fetch("/speaker_ids")
                .then((response) => response.json())
                .then((data) => {
                    speakerSelect.innerHTML = data.join("");
                });

            // 音声認識の結果を受信
            socket.on("SpeechRecognition", (data) => {
                const markdownText = data.text;
                const htmlContent = marked.parse(markdownText);
                chatlog.innerHTML += `<div class="user">${htmlContent}</div>`;
            });

            // AIの応答を受信したときの処理
            socket.on("ai_response", (data) => {
                const markdownText = data.ai_response;
                const htmlContent = marked.parse(markdownText);
                chatlog.innerHTML += `<div class="assistant">${htmlContent}</div>`;
            });

            // 音声ファイルを再生する処理
            socket.on("play_audio", async (data) => {
                const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                const audioUrl = URL.createObjectURL(audioBlob);

                // キューに登録
                audioQueue.push(audioUrl);

                // 再生中でなければ再生
                if (!isPlaying) {
                    playAudio();
                }
                // const audio = new Audio(audioUrl);
                // audio.play();
            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudio() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    return;
                }

                isPlaying = true;
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudio();
                };
            }

            // AIの応答ストリームを受信したときの処理
            socket.on("ai_stream", (data) => {
                if(data.sentens){
                    if (data.sentens.includes("---Start---")) { 
                        // 最初はdivを作成
                        chatlog.innerHTML += `<div class="assistant"></div>`;
                        const assistantDivs = chatlog.getElementsByClassName("assistant");
                        currentDiv = assistantDivs[assistantDivs.length - 1];//作ったdivを取得
                        return;
                    }
                    else if (data.sentens.includes("---End---") ){ 
                        // 終了時はmarkedを適用
                        currentDiv.innerHTML= marked.parse(currentDiv.innerHTML);
                        currentDiv = ""; //初期化
                        return;
                    }
                    else{
                        // sentensをセンテンスキューに登録
                        sentensQueue.push(data.sentens);
                    }
                }

                if(data.audio){
                    // 音声ファイルをキューにと登録
                    const audioBlob = new Blob([data.audio], { type: "audio/mp3" });
                    const audioUrl = URL.createObjectURL(audioBlob);
                    audioQueue.push(audioUrl); // オーディオキューに登録


                    if (!isPlaying) {
                        playAudioWithSentens();
                    }
                }

            });

            // Queueに登録された音声ファイルを再生する処理
            async function playAudioWithSentens() {
                // 再生する音声ファイルがなければ終了
                if (audioQueue.length === 0) {
                    isPlaying = false;
                    return;
                }
                // 再生中フラグを立てる
                isPlaying = true;

                //SentensQueueからセンテンスを取り出して表示
                //ただし、---silent---が含まれている場合は表示しない
                const sentens = sentensQueue.shift();
                if (!sentens.includes("---silent---")){
                    currentDiv.innerHTML += sentens;
                }

                //AudioQueueから音声ファイルを取り出して再生
                const audioUrl = audioQueue.shift();
                const audio = new Audio(audioUrl);
                audio.play();

                // 再生が終了したら次の音声ファイルを再生
                audio.onended = () => {
                    playAudioWithSentens();
                };
            }

            // Spaceキーが押されたときにstartRecordingボタンをクリック
            document.addEventListener("keydown", (event) => {
                if (startRecButton.disabled) {
                    console.log("処理中のため入力はできません");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    startRecButton.click();
                }
            });

            // Spaceキーから指が離されたときにstopRecordingボタンをクリック
            document.addEventListener("keyup", (event) => {
                if (stopRecButton.disabled) {
                    console.log("不正な録音停止操作です");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    stopRecButton.click();
                }
            });

            //Speakerの音声確認テスト
            speakerTestButton.addEventListener("click", () => {
                const speaker = speakerSelect.value;
                fetch("/speaker_test", {
                    method: "POST",
                    headers: {
                        "Content-Type": "application/json",
                    },
                    body: JSON.stringify({ speaker }),
                })
                    .then((response) => response.json())
                    .then((data) => {
                        console.log(data);
                    });
            });

            // 録音開始ボタンがクリックされたときの処理
            startRecButton.addEventListener("click", () => {
                audioContext = new AudioContext();
                const source = audioContext.createMediaStreamSource(window.stream);
                recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
                recorder.record();

                // ボタンを無効化
                setBtnonStart();
            });

            // 録音停止ボタンがクリックされたときの処理
            stopRecButton.addEventListener("click", () => {
                // ボタンを無効化
                setBtnunderProcessing();

                // 録音を停止
                recorder.stop();

                // 録音した音声をファイルに保存して送信
                recorder.exportWAV((blob) => {
                    audioBlob = blob;
                    if (!audioBlob) {
                        console.error("No audio to upload");
                        return;
                    }

                    const formData = new FormData();
                    formData.append("file", audioBlob, "recorded_audio.wav");

                    const speaker = speakerSelect.value;
                    formData.append("speaker", speaker);

                    const method = document.querySelector('input[name="Method"]:checked').value;

                    fetch(method, {
                        method: "POST",
                        body: formData,
                    })
                        .then((response) => response.json())
                        .then((data) => {
                            console.log(data);
                            // ボタン状態の初期化
                            setBtnonRestart();
                        })
                        .catch((error) => {
                            console.error("Upload failed:");
                            // ボタン状態の初期化
                            setBtnonRestart();
                        });
                });
            });

            // ボタン状態の初期化
            setBtnonRestart();
        });

        // ページを離れるときにストリームを停止
        window.addEventListener("beforeunload", () => {
            if (window.stream) {
                window.stream.getTracks().forEach((track) => {
                    track.stop();
                });
            }
        });

    </script>
</body>

</html>

Writing static/index15.html
